In [ ]:
# FUNGSI KONVERSI JAVASCRIPT KE OPENCV
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # MENGKODE GAMBAR BASE64
  image_bytes = b64decode(js_reply.split(',')[1])
  # KONVERSI KE ARRAY
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # MENGKODE ARRAY KE OPENCV BGR
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# FUNGSI CONVERT OPENCV BOUNDING BOX
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # KONVERSI ARRAY KE PIL
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # FORMAT BOUNDING BOX KE PNG
  bbox_PIL.save(iobuf, format='png')
  # FORMAT RETURN STRING
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
# KLASIFIKASI CASCADE ADABOOST
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // RESIZE CITRA KAMERA
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // KLIK CAPTURE
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # SIMPAN DATA CAPTURE
  data = eval_js('takePhoto({})'.format(quality))
  # OPENCV KE FORMAT GAMBAR
  img = js_to_image(data) 
  ## PREPROESSING GRAYSCALE
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  print(gray.shape)
  ## DETEKSI WAJAH DENGAN BOUNDING BOX CASCADE
  faces = face_cascade.detectMultiScale(gray)
  # MENYESUAIKAN BENTUK WAJAH
  for (x,y,w,h) in faces:
      img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
  # SIMPAN GAMBAR
  cv2.imwrite(filename, img)

  return filename

In [ ]:
try:
  filename = take_photo('photo.jpg')
  print('Saved to {}'.format(filename))
  
  # MEMPERLIHATKAN GAMBAR YANG TELAH DI CAPTURE
  display(Image(filename))
except Exception as err:

  print(str(err))